In [41]:
# Boilerplate at top of file
%load_ext autoreload
%load_ext rmagic
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime
import math

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from ggplot import *

#####
# custom functions
#####
import etl
from function import *

#####
# custom to script
####
from sklearn import linear_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic


In [54]:
df_set = etl.etl('competition_data')
print df_set.shape
Y_KEY = 'cost'

(30213, 53)


In [55]:
df_set.columns.values

array(['tube_assembly_id', 'material_id', 'diameter', 'wall', 'length',
       'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x',
       'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other',
       'spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7',
       'spec8', 'spec9', 'spec10', 'component_id_1', 'quantity_1',
       'component_id_2', 'quantity_2', 'component_id_3', 'quantity_3',
       'component_id_4', 'quantity_4', 'component_id_5', 'quantity_5',
       'component_id_6', 'quantity_6', 'component_id_7', 'quantity_7',
       'component_id_8', 'quantity_8', 'end_a_forming', 'end_x_forming',
       'supplier', 'quote_date', 'annual_usage', 'min_order_quantity',
       'bracket_pricing', 'quantity', 'cost', 'component_count',
       'spec_count'], dtype=object)

In [56]:
from sklearn import cross_validation
# Split training and test
df_train, df_test = cross_validation.train_test_split(df_set)
df_train_x = df_train.drop(Y_KEY, 1)
df_train_y = df_train[Y_KEY]
# Testing
df_test_x = df_test.drop(Y_KEY, 1)
df_test_y = df_test[Y_KEY]

In [77]:
df_set.groupby(['end_a']).size()

end_a
EF-001       10
EF-002       45
EF-003    17624
EF-004        2
EF-005       10
EF-006        9
EF-007        8
EF-008     5637
EF-009      607
EF-010        6
EF-011        6
EF-012      226
EF-013       21
EF-014        4
EF-015      104
EF-016       15
EF-017      791
EF-018     3590
EF-019       13
EF-020        1
EF-021       57
EF-022        4
EF-023      494
EF-025        3
NONE        926
dtype: int64

# Some Basic Linear Model

In [60]:
lm = linear_model.LinearRegression()
feature_keys = ['wall', 'length', 'diameter','component_count']
lm.fit(df_train_x[feature_keys], df_train_y)
predict_y = lm.predict(df_test_x[feature_keys])
print metric(predict_y, df_test_y)

nan


# Some Test Random Forest

In [39]:
from sklearn import tree
mtree = tree.DecisionTreeRegressor()
mtree.fit(df_train_x[feature_keys], df_train_y)
predict_y_tree  = mtree.predict(df_test_x[feature_keys])
mtree.score
print metric(predict_y_tree, df_test_y)

0.830599228171


# R Analysis

In [71]:
%%R -i df_set 
m1 <- lm(log(cost) ~ spec_count + component_count  + spec_count+ quantity + bracket_pricing  + wall + bend_radius, data = df_set)
summary(m1)
# qqnorm(m1.rstand)
# qqline(m1.rstand)


Call:
lm(formula = log(cost) ~ spec_count + component_count + spec_count + 
    quantity + bracket_pricing + wall + bend_radius, data = df_set)

Residuals:
    Min      1Q  Median      3Q     Max 
-2.8747 -0.5599 -0.1253  0.5016 10.5530 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)         1.5148971  0.0188628  80.312  < 2e-16 ***
spec_count          0.0180708  0.0026484   6.823 9.07e-12 ***
component_count     0.0361582  0.0054324   6.656 2.86e-11 ***
quantity           -0.0049736  0.0000661 -75.245  < 2e-16 ***
bracket_pricingYes  0.0636935  0.0143016   4.454 8.47e-06 ***
wall                0.4124119  0.0075470  54.646  < 2e-16 ***
bend_radius         0.0001041  0.0000231   4.506 6.64e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.802 on 30206 degrees of freedom
Multiple R-squared:  0.2471,	Adjusted R-squared:  0.2469 
F-statistic:  1652 on 6 and 30206 DF,  p-value: < 2.2e-16

